In [29]:
#export
from IPython.display import Latex
from pytex.helpers import varArgFunc

In [30]:
"NOTE: need to add eager execution, it will definitely help"

'NOTE: need to add eager execution, it will definitely help'

In [31]:
" overall wrap is working perfectly right now. does look like there is any problem"

"""
# this is not needed for now, just will need to implement isRoot for end, to wrap finally in
# gather
if isRoot: build should force env and return Latex(builtString)
there should be only one root in the tree

if forceEnv: build should force the environment but send it string form
"""

'\n# this is not needed for now, just will need to implement isRoot for end, to wrap finally in\n# gather\nif isRoot: build should force env and return Latex(builtString)\nthere should be only one root in the tree\n\nif forceEnv: build should force the environment but send it string form\n'

In [32]:
#export
OPERATOR = {
    "ADD": (' + ', 1),
    "SUB": (' - ', 1),
    "MUL": (' * ', 2),
    "DIV": (' ', 2), 
    "POW": ('^', 3),
    "EQ" : (' = ', 0),
    "NE" : (r' \neq ', 0),
    "OR" : (r'\hspace{1mm}', 0),
    "GT" : (' > ', 0),
    "LT" : (' < ', 0),
    "GE" : (r' \geq ', 0),
    "LE" : (r' \leq ', 0),
    "MAX": ('', 4)
}
def getPrec(operator):
    if str(operator) in OPERATOR: return OPERATOR[str(operator)][1]
    else: return OPERATOR["MAX"][1]
    
def getMid(operator): 
    if str(operator) in OPERATOR: return OPERATOR[str(operator)][0]
    else: return OPERATOR["MAX"][0]

In [41]:
#export
class Var():
    def __init__(self, name, latex=None):
        self._name = str(name)
        self._fn, self._latex = None, None
        self._isTopLevel = True
    
    @property
    def name(self): return self._name
    @name.setter
    def name(self, n): self._name = n
    def setName(self, n): 
        self._name = n
        return self
    
    @property
    def back(self): return self._fn
    @back.setter
    def back(self, fn): self._fn = fn
      
    def __str__(self): return self._name
    def __repr__(self): return self.__str__()
    
    def apply(self, f, *args):
        n = Var('temp')
        a = makeVar(*args)
        n.back = f(*a) if not isinstance(a,Var) else f(a)
        return n
    
    def __add__(self, other):
        return self.apply(BinaryFunc, 'ADD', self, other)
    def __sub__(self, other):
        return self.apply(BinaryFunc, 'SUB', self, other)
    def __mul__(self, other):
        return self.apply(BinaryFunc, 'MUL', self, other)
    def __pow__(self, other):
        return self.apply(BinaryFunc, 'POW', self, other)
    def __or__(self, other):
        return self.apply(BinaryFunc, 'OR', self, other)
    
    def __lt__(self, other):
        return self.lt(other)
    def __gt__(self, other):
        return self.gt(other)
    def __le__(self, other):
        return self.le(other)
    def __ge__(self, other):
        return self.ge(other)
    def __eq__(self, other):
        return self.eq(other)
    def __ne__(self, other):
        return self.ne(other)
    
    def equals(self, other):
        return self.apply(BinaryFunc, 'EQ', self, other)
    def not_equals(self, other):
        return self.apply(BinaryFunc, 'NE', self, other)
    def eq(self,other):
        return self.equals(other)
    def ne(self,other):
        return self.not_equals(other)
    def lt(self, other):
        return self.apply(BinaryFunc, 'LT', self, other)
    def gt(self, other):
        return self.apply(BinaryFunc, 'GT', self, other)
    def le(self, other):
        return self.apply(BinaryFunc, 'LE', self, other)
    def ge(self, other):
        return self.apply(BinaryFunc, 'GE', self, other)
    
    def __neg__(self):
        return self.apply(NegateFunc, self)
    def root(self, other):
        return self.apply(RootFunc, self, other)
    def __truediv__(self, other):
        return self.apply(FracFunc, self, other)
    
    def build(self):
        if self._latex is not None: return self._latex
        else:
            self._latex = self.name if self._fn is None else self._fn.build()
            return self._latex
        
    def func_appl(self, *others):
        return self.apply(FuncApplication, self, *others)
    def __call__(self, *others):
        return self.apply(FuncApplication, self, *others)

def makeVar(*args):
    return varArgFunc(lambda arg: arg if isinstance(arg, Var) else Var(str(arg)), *args)

def addEnv(s, env='gather'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env

def func(f, *args):
    f = makeVar(f)
    args = makeVar(*args)
    return f.func_appl(*args)

def wrap(v):
    return Latex(addEnv(v.build()))

In [42]:
#export
class BasicFunc():
    def __init__(self, operator):
        self.precedence = getPrec(operator)

In [43]:
#export
class BinaryFunc(BasicFunc):
    def __init__(self, mid, a1, a2):
        super().__init__(mid)
        self._mid = getMid(mid)
        self.a1, self.a2 = a1, a2
    @property
    def mid(self):
        return self._mid
    def build(self):
        if self.mid is None:
            raise Exception('middle operator string not set')
        a1, a2 = self.a1.build(), self.a2.build()
        if self.a1.back is not None and self.a1.back.precedence < self.precedence:
            a1 = '(' + a1 + ')'
        if self.a2.back is not None and self.a2.back.precedence < self.precedence:
            a2 = '(' + a2 + ')'
        return f'{a1}{self.mid}{a2}'
                
class FracFunc(BasicFunc):
    def __init__(self, num, den):
        super().__init__('DIV')
        self.num, self.den = num, den
    def build(self):
        return (r"\frac{" + 
                self.num.build() + r"}{" + self.den.build() + r"}")
class RootFunc(BasicFunc):
    def __init__(self, num, root):
        super().__init__('MAX')
        self.num, self.root = num, root
    def build(self):
        return(r"\sqrt[\leftroot{-1}\uproot{1}" + self.root.build() 
               + r"]{" + self.num.build() + r"}")

class NegateFunc(BasicFunc):
    def __init__(self, v):
        super().__init__('MAX')
        self.v = v
    def build(self):
        return f'-{self.v.build()}'
    
class IntegralFunc(BasicFunc):
    def __init__(self, lower, upper, func):
        super().__init__('MAX')
        self.lower, self.upper, self.func = lower, upper, func
    def build(self):
        pass
    
class FuncApplication(BasicFunc):
    def __init__(self, f, *args):
        super().__init__('MAX')
        self.f = f
        if len(args) == 0: self.args = None
        elif isinstance(args, tuple): self.args = list(args)
        else: raise Exception('invalid type passed')
        
    def build(self):
        if self.args is None: return self.f.build() + '()'
        s = self.f.build() + '('
        for arg in self.args:
            (arg)
            s += arg.build() + ','
        s = s[:-1]
        return s + ')'

In [44]:
from pytex.platforms.jupyter import latex
a,b,c,d = makeVar('a','b','c','d')
k = latex(a.le(-b))
l = latex(c.not_equals(d))

{{k}} {{l}}

In [48]:
a,b,c,d = makeVar('a','b','c','d')
latex(c.ne((a+a-b)*(a**2) | (b+b)**2).ne(c))

<IPython.core.display.Latex object>

In [49]:
#export
class Vector(Var):
    def __init__(self, name):
        super().__init__(r"\vec{\mathbf{" + name +"}}")

def makeVector(*args):
    return varArgFunc(lambda arg: arg if isinstance(arg, Vector) else Vector(str(arg)), *args)

In [50]:
#export
class MatrixBuilder():
    def __init__(self):
        self.matrix, self.cols = [], None
        
    def add(self, *args):
        if self.cols is None:
            self.cols = len(args)
        elif not self.cols == len(args):
            raise Exception(f'Different dimensions for matrix builder. dim1:{self.cols} dim2:{len(args)}')
        self.matrix.append(list(args))
        return self
        
    def create(self, subscript=None, power=None, surround='[]'):
        return Matrix(self.matrix, subscript, power, surround)

In [52]:
#export
class Matrix(Var):
    def __init__(self, matrix=[[]], subscript=None, power=None, surround='[]'):
        super().__init__('matrix')
        self.matrix, self.surround = matrix, surround
        self.power = makeVar(power) if power is not None else power
        self.subscript = makeVar(subscript) if subscript is not None else subscript
        
    def __len__(self):
        return self.rows
    
    def __getitem__(self, ind):
        if ind  >= len(self.matrix): raise Exception('index out of bounds')
        if len(self.matrix) == 0: raise Exception('empty matrix') 
        if isinstance(self.matrix[0], list): return Matrix(self.matrix[ind])
        else: return Var(self.matrix[ind])
    
    @staticmethod
    def builder():
        return MatrixBuilder()
        
    @property
    def T(self):
        if len(self.matrix) == 0: return self
        elif(not isinstance(self.matrix[0], list)):
            rows, cols = len(self.matrix), 1
            matrix = [[ele] for ele in self.matrix]
        else:
            rows, cols = len(self.matrix), len(self.matrix[0])
            matrix = [[self.matrix[j][i] for j in range(rows)] for i in range(cols)]
        return Matrix(matrix, self.subscript, self.power, self.surround)
        
    def build(self): 
        env = (r'{bmatrix}' if self.surround == '[]' 
               else r'{pmatrix}' if self.surround == '()'
               else r'{vmatrix}' if self.surround == '||'
               else r'{Vmatrix}' if (self.surround == '|| ||' or self.surround == '||||')
               else r'{matrix}')
        begin = r'\begin' + env
        end = r'\end' + env
        if len(self.matrix) == 0: return begin + end
        
        s = ''
        if not isinstance(self.matrix[0], list):
            # it is a vector
            for ele in self.matrix:
                ele = makeVar(ele)
                s += ' ' + ele.build() + ' & '
            s = s[:-2]
        else:
            for row in self.matrix:
                for ele in row:
                    ele = makeVar(ele)
                    s += ' ' + ele.build() + ' & '
                s = s[:-2] + r"\\"
            s = s[:-2]
        p = '^{' + self.power.build() + '}' if self.power is not None else ''
        subs = '_{' + self.subscript.build() + '}' if self.subscript is not None else ''
        return (r"" + begin + s + end  + p + subs + r"")


In [53]:
#export
class Dots(Var):
    def __init__(self, direc='h'):
        super().__init__(direc + 'dots')
        self.direc = direc
    def build(self):
        return (r'\cdots' if (self.direc == 'h' or self.direc == 'c') 
                else r'\vdots' if (self.direc == 'v') else r'\ddots')

In [54]:
#export
def hasHoriDots(elements, m):
    elerowlen = max([len(ele) for ele in elements]) 
    return not elerowlen == len(m[0])
def hasVertDots(elements, m):
    return not len(elements) == len(m)

class MatrixWithDots(Matrix):
    def __init__(self, elements, shape=None, subscript=None, power=None, surround='[]'):
        # resolve elements
        self.elements, self.shape = elements,shape
        if(not isinstance(elements, list) or len(elements) == 0
          or not isinstance(elements[0], list)): 
            raise Exception('need to pass list of list')
        # resolve shape
        minrows, mincols = len(elements), max([len(row) for row in elements])
        if(shape is None):
            self.rows = minrows + 1 if minrows != 1 else minrows
            self.cols = mincols + 1 if mincols != 1 else mincols
        else:
            try:
                self.rows, self.cols = shape
                self.rows, self.cols = int(self.rows), int(self.cols)
            except:
                raise Exception('shape should be of type tuple(int,int)')
            if self.rows < minrows or self.cols < mincols:
                raise Exception('shape >= (row, cols) of elements')
        m = [[0]*self.cols for i in range(self.rows)]
        i=0
        while i < len(elements)-1:
            row = elements[i]
            j=0
            while(j < len(row)-1):
                m[i][j] = row[j]
                j += 1
            while(j < len(m[i])-1):
                m[i][j] = Dots('h')
                j += 1
            m[i][j] = row[len(row)-1]
            i += 1

        while i < len(m)-1:
            m[i] = [Dots('v')]*len(m[i])
            i += 1

        row = elements[len(elements)-1]
        j=0
        while(j < len(row)-1):
            m[i][j] = row[j]
            j += 1
        while(j < len(m[i])-1):
            m[i][j] = Dots('h')
            j += 1
        m[i][j] = row[len(row)-1]

        dli, dlj = len(m)-2, len(m[0])-2

        if isinstance(m[dli][dlj], Dots) and hasHoriDots(elements, m) and hasVertDots(elements, m):
            m[dli][dlj] = Dots('d')
        super().__init__(m, subscript, power, surround)
        
    @property
    def T(self):
        raise Exception('transpose of dotted matrix not allowed')

In [55]:
latex(MatrixWithDots([[1,2],[2,3],[2,3]], (5,4), surround='||'))

<IPython.core.display.Latex object>

In [56]:
latex(MatrixWithDots([[1,2,3,4],[2,3,4,5]]))

<IPython.core.display.Latex object>

In [57]:
!./notebook2script.py ./Variables.ipynb

Converted Variables.ipynb to pytex/Variables.py
